# Mini Batches
Iterate over a data stream in mini batches

In [ ]:
import pandas as pd
from river import datasets
from river_torch import classification
import torch
from river import compose
from river import preprocessing
from itertools import islice

In [ ]:
dataset = datasets.Phishing()

In [ ]:
def build_fn(n_features, n_classes=1):
    net = torch.nn.Sequential(
        torch.nn.Linear(n_features, n_classes),
        torch.nn.Sigmoid()
    )
    return net

def batcher(iterable, batch_size):
    iterator = iter(iterable)
    while batch := list(islice(iterator, batch_size)):
        yield batch

In [ ]:
model = compose.Pipeline(
    preprocessing.StandardScaler(),
    classification.Classifier(build_fn=build_fn,loss_fn="binary_cross_entropy",optimizer_fn="sgd")
)
model

In [ ]:
for batch in batcher(dataset,5):
    x,y = zip(*batch)
    x = pd.DataFrame(x)
    y = list(y)
    y_pred = model.predict_proba_many(X=x)
    model = model.learn_many(x, y)    # make the model learn